In [1]:
# Import Library
import pandas as pd
import numpy as np

## Load data

In [ ]:

# Load fatal crashes data
# Load data from a specific sheet named "BITRE_Fatal_Crash",skipping non-data rows
crash_df = pd.read_excel(
    'bitre_fatal_crashes_dec2024.xlsx', 
    sheet_name='BITRE_Fatal_Crash',
    skiprows=4
)

## Check Na

In [ ]:
# Define a function to count missing values in a Series
def count_missing(series):
    # Treat actual NaN as missing
    mask = series.isna()
    # Treat blank strings, 'Unknown', 'Undetermined' as missing
    mask |= series.astype(str).str.strip().isin(['', 'Unknown', 'Undetermined'])
    # Treat numeric -9 as missing
    if pd.api.types.is_numeric_dtype(series):
        mask |= (series == -9)
    return mask.sum()

# Apply to every column
missing_counts = crash_df.apply(count_missing)

# Print counts per column, also check col name
print(missing_counts.sort_values(ascending=False))

## Load fatalities data

In [ ]:
fatalities_df = pd.read_excel(
    'bitre_fatalities_dec2024.xlsx', 
    sheet_name='BITRE_Fatality',
    skiprows=4
)

missing_counts_fatalities = fatalities_df.apply(count_missing)

# Print all na counts per column
print(missing_counts_fatalities.sort_values(ascending=False))

## Dimension Tables

### 1.Time 

In [ ]:
# update only keep time data now, keep month, year and weekday in fact table
# there are 39 blank for time, 40 unknown for time_of_day. and they have no location data -> remove directly from raw dataset, same for fatalities dataset(43 rows)

crash_df['Time'] = crash_df['Time'].astype(str).str.strip()
crash_df['Time'] = pd.to_datetime(crash_df['Time'], format='%H:%M:%S', errors='coerce').dt.hour
crash_df['Time'] = crash_df['Time'].astype(int)

# 2. Select 'Time', drop duplicates，sort
dim_time_df = crash_df[['Time']].drop_duplicates().sort_values(by='Time', ascending=True).reset_index(drop=True)

# Add 'Time_ID' column (starting from 1)
dim_time_df.insert(0, 'Time_ID', range(0, len(dim_time_df)))

def get_time_of_day(hour):
    if 6 <= hour <= 18:
        return 'Day'
    elif 0 <= hour <= 5 or 19 <= hour <= 23:
        return 'Night'
    else:
        return 'Unknown'

dim_time_df['Time of Day'] = dim_time_df['Time'].apply(get_time_of_day)
# 5. Define time period based on hour
def get_time_period(hour):
    if 0 <= hour <= 3:
        return 'Midnight'
    elif 4 <= hour <= 7:
        return 'Early Morning'
    elif 8 <= hour <= 11:
        return 'Morning'
    elif 12 <= hour <= 15:
        return 'Afternoon'
    elif 16 <= hour <= 19:
        return 'Evening'
    elif 20 <= hour <= 23:
        return 'Late Night'
    else:
        return 'Unknown'

dim_time_df['Time Period'] = dim_time_df['Time'].apply(get_time_period)

# Export to CSV without the DataFrame index
dim_time_df.to_csv('dim_time.csv', index=False)

print(f" {len(dim_time_df)} unique time combinations.")
print(dim_time_df.head())

 24 unique time combinations.
   Time_ID  Time Time of Day    Time Period
0        0     0       Night       Midnight
1        1     1       Night       Midnight
2        2     2       Night       Midnight
3        3     3       Night       Midnight
4        4     4       Night  Early Morning


### 2.State 

In [ ]:
# instead put all location details in one dimension, i separated into state and remoteness, keep iga in fact table and ignore sa4
dim_state_df = crash_df[['State']].copy().drop_duplicates()

# 3. Reset index after dropping duplicates
dim_state_df = dim_state_df.reset_index(drop=True)

# 4. Add 'Time_ID' column (starting from 1)
dim_state_df.insert(0, 'State_ID', range(1, len(dim_state_df) + 1))

# 5. Export to CSV without the DataFrame index
dim_state_df.to_csv('dim_state.csv', index=False)

print(f" {len(dim_state_df)} unique time combinations.")
print(dim_state_df.head())

 8 unique time combinations.
   State_ID State
0         1   NSW
1         2   Tas
2         3   Vic
3         4   Qld
4         5    SA


### 3.Vehicle involvement

In [26]:

# Extract relevant fields related to vehicle involvement
vehicle_df = crash_df[['Bus \nInvolvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']].copy()

# Rename columns to standardized names
vehicle_df.columns = ['Bus_Involved', 'Heavy_Truck', 'Articulated_Truck']

# yes 1, no 0, -9 ua
vehicle_df.replace({'Yes': 1, 'No': 0}, inplace=True)
vehicle_df = vehicle_df.infer_objects(copy=False)

# Create description column based on combinations
def describe_vehicle(row):
    desc_parts = []

    for label, value in {
        'Bus': row['Bus_Involved'],
        'Heavy rigid truck': row['Heavy_Truck'],
        'Articulated truck': row['Articulated_Truck']
    }.items():
        if value == 1:
            desc_parts.append(f"{label} involved")
        elif value == 0:
            desc_parts.append(f"{label} not involved")
        elif value == -9:
            desc_parts.append(f"{label} involvement unknown")
        else:
            desc_parts.append(f"{label} value invalid")

    return "; ".join(desc_parts)


vehicle_df['Description'] = vehicle_df.apply(describe_vehicle, axis=1)
# Remove duplicate records to build a unique dimension table
dim_vehicle_df = vehicle_df.drop_duplicates().reset_index(drop=True)

# Add a surrogate primary key
dim_vehicle_df.insert(0, 'Vehicle_ID', range(1, len(dim_vehicle_df) + 1))

# Optional: save the dimension table to CSV
dim_vehicle_df.to_csv('dim_vehicle.csv', index=False)

# Display the resulting vehicle dimension table
dim_vehicle_df.head()


/var/folders/sf/97mgdx1j24nbpzp1_h96gl_40000gn/T/ipykernel_9504/745254592.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  vehicle_df.replace({'Yes': 1, 'No': 0}, inplace=True)


,Vehicle_ID,Bus_Involved,Heavy_Truck,Articulated_Truck,Description
0,1,0,0,0,Bus not involved; Heavy rigid truck not involv...
1,2,-9,-9,-9,Bus involvement unknown; Heavy rigid truck inv...
2,3,0,0,1,Bus not involved; Heavy rigid truck not involv...
3,4,1,0,1,Bus involved; Heavy rigid truck not involved; ...
4,5,0,1,0,Bus not involved; Heavy rigid truck involved; ...


### 4.Road Type

In [27]:

# Extract the relevant column for road type
road_df = crash_df[['National Road Type']].copy()

# Rename the column for consistency
road_df.columns = ['Road_Type']

# Remove rows with missing values and drop duplicates
road_df = road_df.dropna().drop_duplicates().reset_index(drop=True)

# Add a surrogate primary key
road_df.insert(0, 'Road_ID', range(1, len(road_df) + 1))

# Optional: save the dimension table to CSV
road_df.to_csv('dim_road_type.csv', index=False)

# Display the resulting road type dimension table
road_df.head()


,Road_ID,Road_Type
0,1,Arterial Road
1,2,Local Road
2,3,National or State Highway
3,4,Undetermined
4,5,Sub-arterial Road


### 5.Holiday flag

In [28]:

# Extract holiday-related columns
holiday_df = crash_df[['Christmas Period', 'Easter Period']].copy()

# Rename columns for clarity
holiday_df.columns = ['Christmas_Flag', 'Easter_Flag']

# Remove rows with missing values and drop duplicate combinations
holiday_df = holiday_df.dropna().drop_duplicates().reset_index(drop=True)

# Add a surrogate primary key
holiday_df.insert(0, 'Holiday_ID', range(1, len(holiday_df) + 1))

# Optional: save to CSV
holiday_df.to_csv('dim_holiday.csv', index=False)

# Display the result
holiday_df.head()


,Holiday_ID,Christmas_Flag,Easter_Flag
0,1,Yes,No
1,2,No,No
2,3,No,Yes


### 6.Speed Limit

In [36]:
# Extract the speed limit column
speed_df = crash_df[['Speed Limit']].copy()

# Rename column for consistency
speed_df.columns = ['Speed_Value']

# Replace special values
# '<40' is a string, replace it with 35
# -9 is used for unknown values, replace with 0
speed_df['Speed_Value'] = speed_df['Speed_Value'].replace({'<40': 35, -9: 0})

# Convert to integer
speed_df['Speed_Value'] = speed_df['Speed_Value'].astype(int)

# Drop duplicates
speed_df = speed_df.drop_duplicates().reset_index(drop=True)
speed_df = speed_df.sort_values(by='Speed_Value').reset_index(drop=True)

# Add surrogate key
speed_df.insert(0, 'Speed_ID', range(1, len(speed_df) + 1))

# Add Description based on speed limit values
speed_df['Description'] = speed_df['Speed_Value'].apply(
    lambda x: 'Unknown' if x == 0 else ('<40 km/h' if x == 35 else f'{x} km/h')

)

# Optional: save to CSV
speed_df.to_csv('dim_speed_limit.csv', index=False)

# Preview
speed_df.head()


/var/folders/sf/97mgdx1j24nbpzp1_h96gl_40000gn/T/ipykernel_1012/1915951888.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  speed_df['Speed_Value'] = speed_df['Speed_Value'].replace({'<40': 35, -9: 0})


,Speed_ID,Speed_Value,Description
0,1,0,Unknown
1,2,5,5 km/h
2,3,10,10 km/h
3,4,15,15 km/h
4,5,20,20 km/h


### 7.Crash Type

In [32]:

# Extract the crash type column
crash_type_df = crash_df[['Crash Type']].copy()

# Rename column for consistency
crash_type_df.columns = ['Crash_Type']

crash_type_df = crash_type_df.drop_duplicates().reset_index(drop=True)

# Add a surrogate primary key
crash_type_df.insert(0, 'Crash_Type_ID', range(1, len(crash_type_df) + 1))

# Optional: save to CSV
crash_type_df.to_csv('dim_crash_type.csv', index=False)

# Display the result
crash_type_df.head()


,Crash_Type_ID,Crash_Type
0,1,Single
1,2,Multiple


### 8.National Remoteness Areas

In [37]:
remoteness_df = crash_df[['National Remoteness Areas']].copy()

# Rename column for consistency
remoteness_df.columns = ['Remoteness_Areas']
remoteness_df['Remoteness_Areas'] = remoteness_df['Remoteness_Areas'].astype(str).str.strip()

# Mark blank or 'Unknown' as 'Unknown'
remoteness_df['Remoteness_Areas'] = remoteness_df['Remoteness_Areas'].replace(['', 'Unknown'], 'Unknown')

# Separate unknown and known values
unknown_df = pd.DataFrame([['Unknown']], columns=['Remoteness_Areas'])
known_df = remoteness_df[remoteness_df['Remoteness_Areas'] != 'Unknown'].drop_duplicates()

# Combine with 'Unknown' on top
final_df = pd.concat([unknown_df, known_df.drop_duplicates()]).reset_index(drop=True)

final_df.insert(0, 'Remoteness_ID', range(len(final_df)))

final_df.to_csv('dim_remoteness.csv', index=False)
print(final_df)

   Remoteness_ID           Remoteness_Areas
0              0                    Unknown
1              1   Inner Regional Australia
2              2   Outer Regional Australia
3              3  Major Cities of Australia
4              4      Very Remote Australia
5              5           Remote Australia


## Fact Tables

### Fact Table 1：Fact_Crash

In [44]:
import pandas as pd

crash_df = pd.read_excel('bitre_fatal_crashes_dec2024.xlsx', sheet_name='BITRE_Fatal_Crash', skiprows=4)

# select columns and rename
fact_crash_data = crash_df.rename(columns={
    'Crash ID': 'Crash_ID',
    'State': 'State',
    'National Remoteness Areas': 'Remoteness_area',
    'National Road Type': 'Road_Type',
    'National LGA Name 2021': 'LGA_name',
    'Crash Type': 'Crash_Type',
    'Speed Limit': 'Speed_Value',
    'Christmas Period': 'Christmas_Flag',
    'Easter Period': 'Easter_Flag',
    'Bus \nInvolvement': 'Bus_Involved',
    'Heavy Rigid Truck Involvement': 'Heavy_Truck',
    'Articulated Truck Involvement': 'Articulated_Truck'
})[
    ['Crash_ID', 'State', 'Remoteness_area', 'Road_Type', 'Month', 'Year', 'Dayweek', 'LGA_name', 'Number Fatalities', 'Crash_Type','Speed_Value','Christmas_Flag', 'Easter_Flag', 'Bus_Involved', 'Heavy_Truck', 'Articulated_Truck']
].copy()

# Fill blanks with Unknown
fact_crash_data.fillna({
    'LGA_name': 'Unknown',
    'Remoteness_area': 'Unknown'
}, inplace=True)

# identify mapping for State and its id
state_mapping = {
    'NSW': 1,
    'Tas': 2,
    'Vic': 3,
    'Qld': 4,
    'SA': 5,
    'WA': 6,
    'ACT': 7,
    'NT': 8
}
remoteness_mapping = {
    'Unknown': 0,
    'Inner Regional Australia': 1,
    'Outer Regional Australia': 2,
    'Major Cities of Australia': 3,
    'Very Remote Australia': 4,
    'Remote Australia': 5
}

road_type_mapping = {
    'Arterial Road': 1,
    'Local Road': 2,
    'National or State Highway': 3,
    'Undetermined': 4,
    'Sub-arterial Road': 5,
    'Collector Road': 6,
    'Pedestrian Thoroughfare': 7,
    'Access road': 8,
    'Busway': 9
}

crash_type_mapping = {
    'Single': 1, 'Multiple': 2, 'Unknown': 0
}
speed_mapping = {
    0: 4, 5: 12, 10: 11, 15: 17, 20: 10, 25: 16, 30: 14, 35: 15, 40: 9, 50: 3,
    60: 6, 70: 7, 75: 18, 80: 2, 90: 5, 100: 1, 110: 8, 130: 13
}

holiday_map = {
    ('Yes', 'No'): 1,
    ('No', 'No'): 2,
    ('No', 'Yes'): 3
}

# Vehicle involvement mapping
vehicle_map = {
    (0, 0, 0): 1,
    (-9, -9, -9): 2,
    (0, 0, 1): 3,
    (1, 0, 1): 4,
    (0, 1, 0): 5,
    (-9, 1, -9): 6,
    (0, 1, 1): 7,
    (1, 0, 0): 8,
    (1, 1, 0): 9,
    (-9, -9, 1): 10,
    (0, -9, 0): 11,
    (0, -9, 1): 12,
    (1, -9, 0): 13,
    (1, -9, 1): 14
}


# Extract hour from Time column as Time_ID after crash_id
crash_df['Time'] = crash_df['Time'].astype(str).str.strip()
crash_df['Time'] = pd.to_datetime(crash_df['Time'], format='%H:%M:%S', errors='coerce').dt.hour
crash_df['Time'] = crash_df['Time'].fillna(0).astype(int)
fact_crash_data.insert(1, 'Time_ID', crash_df['Time'])


# Replace 'Yes', 'No', and '-9' with respective values
fact_crash_data['Bus_Involved'].replace({'Yes': 1, 'No': 0, '-9': -9}, inplace=True)
fact_crash_data['Heavy_Truck'].replace({'Yes': 1, 'No': 0, '-9': -9}, inplace=True)
fact_crash_data['Articulated_Truck'].replace({'Yes': 1, 'No': 0, '-9': -9}, inplace=True)

# Create Vehicle_ID based on Bus_Involved, Heavy_Truck, and Articulated_Truck columns
fact_crash_data['Vehicle_ID'] = list(zip(fact_crash_data['Bus_Involved'], fact_crash_data['Heavy_Truck'], fact_crash_data['Articulated_Truck']))
fact_crash_data['Vehicle_ID'] = fact_crash_data['Vehicle_ID'].map(vehicle_map)

# Speed value：<40 -> 35，-9 -> 0
fact_crash_data['Speed_Value'] = fact_crash_data['Speed_Value'].replace({'<40': 35, -9: 0}).astype(int)
fact_crash_data.insert(2, 'Speed_ID', fact_crash_data['Speed_Value'].map(speed_mapping).fillna(4).astype(int))

# Map Holiday_ID
fact_crash_data['Holiday_ID'] = list(zip(fact_crash_data['Christmas_Flag'], fact_crash_data['Easter_Flag']))
fact_crash_data['Holiday_ID'] = fact_crash_data['Holiday_ID'].map(holiday_map)

# add other ids 
fact_crash_data['State_ID'] = fact_crash_data['State'].map(state_mapping).fillna(0).astype(int)
fact_crash_data['Remoteness_ID'] = fact_crash_data['Remoteness_area'].map(remoteness_mapping).astype(int)
fact_crash_data['Road_type_ID'] = fact_crash_data['Road_Type'].map(road_type_mapping).astype(int)
fact_crash_data['Crash_type_ID'] = fact_crash_data['Crash_Type'].map(crash_type_mapping).astype(int)


fact_crash_data.drop(columns=['State', 'Remoteness_area','Road_Type', 'Crash_Type','Speed_Value', 'Christmas_Flag', 'Easter_Flag', 'Bus_Involved', 'Heavy_Truck', 'Articulated_Truck'], inplace=True)
#reorder to match db schema
cols = ['Crash_ID', 'Time_ID', 'Speed_ID','State_ID', 'Remoteness_ID', 'Road_type_ID', 'Crash_type_ID','Vehicle_ID','Holiday_ID',
        'Month', 'Year', 'Dayweek', 'LGA_name', 'Number Fatalities']
fact_crash_data = fact_crash_data[cols]
fact_crash_data.to_csv('fact_crash_1.csv', index=False)

print(fact_crash_data.head(10))


   Crash_ID  Time_ID  Speed_ID  State_ID  Remoteness_ID  Road_type_ID  \
0  20241115        4         1         1              1             1   
1  20241125        6         2         1              1             2   
2  20246013        9         3         2              1             2   
3  20241002       10         1         1              2             3   
4  20242261       11         4         3              0             4   
5  20243185       13         1         4              1             3   
6  20244016       13         1         5              2             5   
7  20245001       17         5         6              0             4   
8  20243168       19         6         4              1             2   
9  20246003       19         3         2              1             2   

   Crash_type_ID  Vehicle_ID  Holiday_ID  Month  Year Dayweek  \
0              1           1           1     12  2024  Friday   
1              1           1           2     12  2024  Friday   
2

/var/folders/sf/97mgdx1j24nbpzp1_h96gl_40000gn/T/ipykernel_1012/3373127826.py:102: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fact_crash_data['Bus_Involved'].replace({'Yes': 1, 'No': 0, '-9': -9}, inplace=True)
/var/folders/sf/97mgdx1j24nbpzp1_h96gl_40000gn/T/ipykernel_1012/3373127826.py:102: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_optio

### Fact Table 2：Fact_Fatality


In [54]:
import pandas as pd
fatality_df = pd.read_excel('bitre_fatalities_dec2024.xlsx', sheet_name='BITRE_Fatality', skiprows=4)

# Select and rename the relevant columns for fatalities data
fatalities_data = fatality_df.rename(columns={
    'Crash ID': 'Crash_ID',
    'Road User': 'Road_User',
    'Gender': 'Gender',
    'Age': 'Age',
    'Age Group': 'Age_Group'
})[
    ['Crash_ID', 'Road_User', 'Gender', 'Age', 'Age_Group']
].copy()
fatalities_data['Road_User'] = fatalities_data['Road_User'].str.strip()
fatalities_data['Gender'] = fatalities_data['Gender'].str.strip()
fatalities_data['Age_Group'] = fatalities_data['Age_Group'].str.strip()

# Replace invalid values ('Other' and '-9') with 'Unknown' in 'Road_User', 'Gender', 'Age', and 'Age_Group' columns
fatalities_data['Road_User'] = fatalities_data['Road_User'].replace({'Other/-9': 'Unknown'})
fatalities_data['Gender'] = fatalities_data['Gender'].fillna('Unknown')

fatalities_data['Age_Group'] = fatalities_data['Age_Group'].fillna('Unknown')


# Create a unique fatality_id for each record (this will act as the primary key)
fatalities_data['fatality_id'] = range(1, len(fatalities_data) + 1)

# Rearrange columns to match the required schema
fatalities_data = fatalities_data[['fatality_id', 'Crash_ID', 'Road_User', 'Gender', 'Age', 'Age_Group']]

# Save the resulting DataFrame to a CSV file
fatalities_data.to_csv('fact_fatalities.csv', index=False)

# Print the first few rows to verify the result
print(fatalities_data.head())


   fatality_id  Crash_ID  Road_User  Gender  Age Age_Group
0            1  20241115     Driver    Male   74  65_to_74
1            2  20241125     Driver  Female   19  17_to_25
2            3  20246013     Driver  Female   33  26_to_39
3            4  20241002     Driver  Female   32  26_to_39
4            5  20242261  Passenger    Male   62  40_to_64


In [5]:
# reload speed value
import pandas as pd
update_df = pd.read_excel('bitre_fatal_crashes_dec2024.xlsx', sheet_name='BITRE_Fatal_Crash', skiprows=4)
# Speed value：<40 -> 35，-9 -> 0
update_df['Speed_Value'] = (
    update_df['Speed Limit']
    .replace({'<40': 35, -9: 0})
    .astype(int)
)

speed_mapping = {
     0:  1,
     5:  2,
    10:  3,
    15:  4,
    20:  5,
    25:  6,
    30:  7,
    35:  8,
    40:  9,
    50: 10,
    60: 11,
    70: 12,
    75: 13,
    80: 14,
    90: 15,
   100: 16,
   110: 17,
   130: 18
}

speed_data = pd.DataFrame({
    'Speed_Value': update_df['Speed_Value']
})
speed_data['Speed_ID'] = speed_data['Speed_Value'].map(speed_mapping)
# only keep id
speed_data[['Speed_ID']].to_csv('speed.csv', index=False)

print(speed_data.head())

   Speed_Value  Speed_ID
0          100        16
1           80        14
2           50        10
3          100        16
4            0         1


/var/folders/sf/97mgdx1j24nbpzp1_h96gl_40000gn/T/ipykernel_9322/2713569223.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({'<40': 35, -9: 0})
